# 02 - Auto-sklearn 使用指南

> 🎯 学习目标：理解贝叶斯优化和元学习，掌握 Auto-sklearn 的高级特性

---

## 📚 本节内容

1. Auto-sklearn 原理简介
2. 基础使用方法
3. 贝叶斯优化深入
4. 元学习(Meta-learning)
5. 自动集成学习
6. 与 sklearn 生态集成

---

## 🧠 Auto-sklearn 核心原理

### 三大核心组件

1. **贝叶斯优化** (Bayesian Optimization)
   - 智能搜索超参数空间
   - 比随机搜索/网格搜索更高效

2. **元学习** (Meta-learning)
   - 从历史数据集学习经验
   - 快速找到好的起始点

3. **自动集成** (Ensemble)
   - 自动构建模型集成
   - 提升最终性能

---

## 1. 平台兼容性说明与安装

### ⚠️ 重要提示：平台兼容性

**Auto-sklearn 平台支持情况：**

| 操作系统 | 支持状态 | 说明 |
|---------|---------|------|
| 🐧 Linux | ✅ 完全支持 | 官方推荐平台 |
| 🪟 Windows | ⚠️ 部分支持 | 需要 WSL2 或 Docker |
| 🍎 macOS | ❌ 不支持 | 依赖项不兼容 |

**本教程的解决方案：**
- **理论部分**：讲解 Auto-sklearn 的核心原理（适用所有平台）
- **实践部分**：提供跨平台的 AutoML 替代方案
  - **PyCaret**：功能强大，支持所有平台
  - **TPOT**：基于遗传算法，轻量级
  - **Optuna + sklearn**：灵活的超参数优化

---

### 1.1 安装指南

#### macOS 用户（推荐）

In [ ]:
# ============================================================
# 安装 AutoML 工具（首次运行时执行）
# ============================================================

# macOS 用户：使用 PyCaret（已预装）
# !pip install pycaret

# 其他可选工具
# !pip install tpot              # 基于遗传算法的 AutoML
# !pip install optuna            # 超参数优化框架
# !pip install flaml             # 微软的轻量级 AutoML

# Linux 用户（可选安装 auto-sklearn）
# !pip install auto-sklearn

# ============================================================
# 导入基础库
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# 设置中文显示（可选）
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']  # macOS
plt.rcParams['axes.unicode_minus'] = False

print("✅ 基础库导入成功")
print(f"📊 NumPy 版本: {np.__version__}")
print(f"📊 Pandas 版本: {pd.__version__}")

---

## 2. 分类任务基础使用

### 2.1 加载数据

In [2]:
from sklearn.datasets import load_digits

# 加载手写数字数据集
X, y = load_digits(return_X_y=True)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"训练集形状: {X_train.shape}")
print(f"测试集形状: {X_test.shape}")

训练集形状: (1437, 64)
测试集形状: (360, 64)


### 2.2 Auto-sklearn 原理讲解（理论）

> 💡 **注意**：虽然 Auto-sklearn 在 macOS 上无法直接运行，但理解其核心原理对学习 AutoML 非常重要。
> 
> 下一节我们将使用 **PyCaret** 实现相同功能（支持所有平台）。

#### Auto-sklearn 的三大核心技术

**1. 贝叶斯优化 (Bayesian Optimization)**

传统的超参数搜索方法：
- ❌ **网格搜索**：遍历所有组合，效率低
- ❌ **随机搜索**：随机采样，缺乏智能

贝叶斯优化的优势：
- ✅ 利用历史实验结果指导下一步搜索
- ✅ 平衡"探索"(exploration)和"利用"(exploitation)
- ✅ 在有限时间内找到更优解

**工作原理：**
```
初始化 → 随机尝试几组参数
    ↓
建立代理模型（Surrogate Model）预测参数性能
    ↓
选择最有希望的参数组合进行实验
    ↓
更新代理模型 → 重复
```

**2. 元学习 (Meta-learning)**

- 从 140+ 个历史数据集中学习经验
- 根据当前数据集特征，推荐合适的起始模型
- 避免从零开始搜索，加速收敛

**3. 自动集成 (Ensemble)**

- 自动构建多个模型的集成
- 使用 Ensemble Selection 算法
- 比单一模型更稳定、更准确

---

In [ ]:
### 2.3 跨平台实践：使用 PyCaret（所有系统可用）

# ============================================================
# PyCaret 是一个功能强大的 AutoML 库，支持：
# - ✅ Windows / macOS / Linux 全平台
# - ✅ 30+ 机器学习算法自动对比
# - ✅ 自动特征工程、超参数调优
# - ✅ 可视化分析丰富
# ============================================================

from pycaret.classification import *

# 1. 初始化 PyCaret 环境
# ============================================================
# setup() 函数会自动完成：
#   - 数据预处理（缺失值处理、编码）
#   - 训练集/测试集划分
#   - 特征缩放
#   - 特征工程（可选）
# ============================================================

print("初始化 PyCaret 环境...")

# 将数据转换为 DataFrame（PyCaret 要求）
data = pd.DataFrame(X_train, columns=[f'pixel_{i}' for i in range(64)])
data['target'] = y_train

# 创建 PyCaret 实验环境
# - data: 训练数据
# - target: 目标列名
# - session_id: 随机种子（保证可复现）
# - verbose: 是否显示详细信息
clf_setup = setup(
    data=data, 
    target='target',
    session_id=42,
    verbose=False,  # 静默模式，减少输出
    html=False      # 不生成 HTML 报告
)

print("✅ PyCaret 环境初始化完成！")

### 2.4 自动对比所有模型

In [ ]:
# ============================================================
# compare_models() 会自动：
# 1. 训练所有可用的分类算法（30+ 种）
# 2. 使用交叉验证评估每个模型
# 3. 按准确率排序，返回最佳模型
# ============================================================

print("开始自动对比所有模型...")
print("这可能需要 1-2 分钟，请耐心等待...\n")

# 对比所有模型，返回性能排行榜
# - fold: 交叉验证的折数（默认10折）
# - sort: 排序指标（Accuracy、F1、AUC等）
# - n_select: 返回前 N 个最佳模型
best_models = compare_models(
    fold=5,           # 使用 5 折交叉验证（节省时间）
    sort='Accuracy',  # 按准确率排序
    n_select=3        # 返回前 3 个最佳模型
)

print("\n✅ 模型对比完成！")
print(f"📊 最佳模型: {type(best_models[0]).__name__}")

### 2.5 超参数调优

In [ ]:
# ============================================================
# tune_model() 会对最佳模型进行超参数调优
# 使用的优化算法：
#   - Random Grid Search（随机网格搜索）
#   - 或 Bayesian Optimization（贝叶斯优化）
# ============================================================

print("开始对最佳模型进行超参数调优...")

# 选择第一个最佳模型进行调优
best_model = best_models[0]

# 超参数调优
# - optimize: 优化目标（Accuracy、F1、AUC等）
# - n_iter: 迭代次数（越大效果越好，但耗时更长）
# - search_library: 搜索库（'scikit-learn'、'optuna'、'tune-sklearn'）
tuned_model = tune_model(
    best_model,
    optimize='Accuracy',  # 优化准确率
    n_iter=10,            # 尝试 10 组超参数
    choose_better=True    # 只有更好时才替换
)

print("\n✅ 超参数调优完成！")

### 2.6 预测与评估

In [ ]:
# ============================================================
# 在测试集上评估模型性能
# ============================================================

# 准备测试数据（需要转换为 DataFrame）
test_data = pd.DataFrame(X_test, columns=[f'pixel_{i}' for i in range(64)])
test_data['target'] = y_test

# 使用 predict_model() 在测试集上预测
# 这个函数会：
#   1. 使用训练好的模型进行预测
#   2. 自动计算各种评估指标
#   3. 返回包含预测结果的 DataFrame
predictions = predict_model(tuned_model, data=test_data)

# 提取预测标签（PyCaret 会在 'prediction_label' 列）
y_pred = predictions['prediction_label'].values

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f"📊 测试集准确率: {accuracy:.4f}")

# 打印详细的分类报告
print("\n" + "="*60)
print("详细分类报告:")
print("="*60)
print(classification_report(y_test, y_pred, target_names=[f'Class {i}' for i in range(10)]))

### 2.7 模型可视化分析

In [ ]:
# ============================================================
# PyCaret 提供了丰富的可视化功能
# plot_model() 可以绘制多种分析图表
# ============================================================

print("生成可视化分析...")

# 1. 混淆矩阵（Confusion Matrix）
# 显示每个类别的预测情况
plot_model(tuned_model, plot='confusion_matrix', plot_kwargs={'percent': True})
plt.title('混淆矩阵（百分比）')
plt.tight_layout()
plt.show()

# 2. 类别预测误差（Class Prediction Error）
# 显示每个类别的预测分布
plot_model(tuned_model, plot='class_report')
plt.title('类别预测报告')
plt.tight_layout()
plt.show()

# 3. ROC 曲线（多分类）
# 显示每个类别的 ROC 曲线
plot_model(tuned_model, plot='auc')
plt.title('ROC 曲线（多分类）')
plt.tight_layout()
plt.show()

print("\n✅ 可视化分析完成！")

---

## 3. 回归任务示例

### 3.1 加载回归数据

In [ ]:
# ============================================================
# 回归任务：预测糖尿病病情进展
# ============================================================

from sklearn.datasets import load_diabetes

# 加载糖尿病数据集
# - 特征：年龄、性别、BMI、血压、血清指标等
# - 目标：一年后的疾病进展指标
X_reg, y_reg = load_diabetes(return_X_y=True)

# 划分训练集和测试集
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

print(f"📊 训练集形状: {X_train_reg.shape}")
print(f"📊 测试集形状: {X_test_reg.shape}")
print(f"\n目标变量统计:")
print(f"  均值: {y_train_reg.mean():.2f}")
print(f"  标准差: {y_train_reg.std():.2f}")
print(f"  范围: [{y_train_reg.min():.2f}, {y_train_reg.max():.2f}]")

### 3.2 使用 PyCaret 进行回归任务

In [ ]:
# ============================================================
# PyCaret 回归任务工作流
# ============================================================

from pycaret.regression import *

# 1. 准备数据
data_reg = pd.DataFrame(X_train_reg, columns=[f'feature_{i}' for i in range(10)])
data_reg['target'] = y_train_reg

# 2. 初始化回归环境
print("初始化 PyCaret 回归环境...")
reg_setup = setup(
    data=data_reg,
    target='target',
    session_id=42,
    verbose=False,
    html=False
)

# 3. 对比所有回归模型
print("\n对比所有回归模型...")
best_reg_models = compare_models(
    fold=5,
    sort='R2',      # 按 R² 排序
    n_select=3
)

# 4. 调优最佳模型
print("\n超参数调优...")
tuned_reg_model = tune_model(
    best_reg_models[0],
    optimize='R2',
    n_iter=10
)

# 5. 评估
test_data_reg = pd.DataFrame(X_test_reg, columns=[f'feature_{i}' for i in range(10)])
test_data_reg['target'] = y_test_reg

predictions_reg = predict_model(tuned_reg_model, data=test_data_reg)
y_pred_reg = predictions_reg['prediction_label'].values

# 计算指标
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

r2 = r2_score(y_test_reg, y_pred_reg)
rmse = np.sqrt(mean_squared_error(y_test_reg, y_pred_reg))
mae = mean_absolute_error(y_test_reg, y_pred_reg)

print("\n" + "="*60)
print("回归模型评估结果:")
print("="*60)
print(f"📊 R² Score: {r2:.4f}  (越接近1越好)")
print(f"📊 RMSE: {rmse:.2f}     (越小越好)")
print(f"📊 MAE: {mae:.2f}      (越小越好)")
print("="*60)

---

## 4. 高级特性：AutoML 深度对比

### 4.1 贝叶斯优化 vs 随机搜索（理论对比）

In [ ]:
# ============================================================
# 对比三种超参数搜索策略：
# 1. 网格搜索（Grid Search）- 遍历所有组合
# 2. 随机搜索（Random Search）- 随机采样
# 3. 贝叶斯优化（Bayesian Optimization）- 智能搜索
# ============================================================

import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# 重新加载分类数据
X_cmp, y_cmp = load_digits(return_X_y=True)
X_train_cmp, X_test_cmp, y_train_cmp, y_test_cmp = train_test_split(
    X_cmp, y_cmp, test_size=0.2, random_state=42
)

# 定义超参数空间
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

results = {}

# ========== 1. 网格搜索 ==========
print("🔍 方法1: 网格搜索（Grid Search）")
print("   特点：遍历所有参数组合，效率低但结果确定")
start = time.time()
grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=3,
    n_jobs=-1,
    verbose=0
)
grid_search.fit(X_train_cmp, y_train_cmp)
grid_time = time.time() - start
grid_score = accuracy_score(y_test_cmp, grid_search.predict(X_test_cmp))
results['Grid Search'] = {'time': grid_time, 'accuracy': grid_score}
print(f"   ⏱️  耗时: {grid_time:.2f}秒")
print(f"   🎯 准确率: {grid_score:.4f}\n")

# ========== 2. 随机搜索 ==========
print("🎲 方法2: 随机搜索（Random Search）")
print("   特点：随机采样参数组合，效率高但缺乏智能")
start = time.time()
random_search = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    n_iter=20,  # 只尝试20组参数
    cv=3,
    random_state=42,
    n_jobs=-1,
    verbose=0
)
random_search.fit(X_train_cmp, y_train_cmp)
random_time = time.time() - start
random_score = accuracy_score(y_test_cmp, random_search.predict(X_test_cmp))
results['Random Search'] = {'time': random_time, 'accuracy': random_score}
print(f"   ⏱️  耗时: {random_time:.2f}秒")
print(f"   🎯 准确率: {random_score:.4f}\n")

# ========== 3. PyCaret (使用贝叶斯优化) ==========
print("🧠 方法3: PyCaret（内置智能优化）")
print("   特点：智能搜索，平衡探索与利用")
# (已在前面训练过，这里使用之前的结果)
print(f"   ⏱️  耗时: ~60-120秒（包含模型对比）")
print(f"   🎯 准确率: {accuracy:.4f}\n")

# 可视化对比
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 耗时对比
methods = list(results.keys())
times = [results[m]['time'] for m in methods]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

ax1.barh(methods, times, color=colors)
ax1.set_xlabel('耗时（秒）', fontsize=12)
ax1.set_title('超参数搜索耗时对比', fontsize=14, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# 准确率对比
accuracies = [results[m]['accuracy'] for m in methods]
ax2.barh(methods, accuracies, color=colors)
ax2.set_xlabel('准确率', fontsize=12)
ax2.set_xlim(0.9, 1.0)
ax2.set_title('模型准确率对比', fontsize=14, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 总结:")
print("   ✅ 网格搜索: 结果最优但耗时长")
print("   ⚡ 随机搜索: 效率高但性能不稳定")
print("   🎯 贝叶斯优化: 智能搜索，性能与效率平衡最佳")

### 4.2 模型集成（Ensemble）

In [ ]:
# ============================================================
# 模型集成：组合多个模型提升性能
# 
# PyCaret 提供三种集成方法：
# 1. Bagging - 减少方差，提高稳定性
# 2. Boosting - 减少偏差，提高准确性
# 3. Blending/Stacking - 组合多个不同模型
# ============================================================

# 方法1: Bagging（装袋法）
# 使用 Bootstrap 采样训练多个模型，然后投票
print("🎒 方法1: Bagging（装袋法）")
print("   原理：训练多个相同模型，降低方差")

bagged_model = ensemble_model(
    tuned_model,
    method='Bagging',
    n_estimators=10  # 集成10个模型
)

# 在测试集上评估
bagged_pred = predict_model(bagged_model, data=test_data)
bagged_accuracy = accuracy_score(y_test, bagged_pred['prediction_label'])
print(f"   🎯 Bagging 准确率: {bagged_accuracy:.4f}\n")

# 方法2: Boosting（提升法）
# 逐步训练模型，后续模型关注前面模型的错误
print("🚀 方法2: Boosting（提升法）")
print("   原理：逐步训练，每个模型修正前一个的错误")

boosted_model = ensemble_model(
    tuned_model,
    method='Boosting',
    n_estimators=10
)

boosted_pred = predict_model(boosted_model, data=test_data)
boosted_accuracy = accuracy_score(y_test, boosted_pred['prediction_label'])
print(f"   🎯 Boosting 准确率: {boosted_accuracy:.4f}\n")

# 方法3: Blending（混合法）
# 组合前3个最佳模型
print("🎨 方法3: Blending（混合多个不同模型）")
print("   原理：组合不同算法的优势")

blended_model = blend_models(
    estimator_list=best_models,  # 使用前3个最佳模型
    method='soft'  # 使用概率投票
)

blended_pred = predict_model(blended_model, data=test_data)
blended_accuracy = accuracy_score(y_test, blended_pred['prediction_label'])
print(f"   🎯 Blending 准确率: {blended_accuracy:.4f}\n")

# 可视化对比
ensemble_results = {
    '单一模型': accuracy,
    'Bagging': bagged_accuracy,
    'Boosting': boosted_accuracy,
    'Blending': blended_accuracy
}

plt.figure(figsize=(10, 6))
methods = list(ensemble_results.keys())
accuracies = list(ensemble_results.values())
colors = ['#95E1D3', '#F38181', '#AA96DA', '#FCBAD3']

bars = plt.barh(methods, accuracies, color=colors)
plt.xlabel('准确率', fontsize=12)
plt.xlim(0.95, 1.0)
plt.title('集成方法性能对比', fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)

# 添加数值标签
for i, (bar, acc) in enumerate(zip(bars, accuracies)):
    plt.text(acc + 0.001, i, f'{acc:.4f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

print("💡 集成学习总结:")
print("   ✅ Bagging: 适合降低模型方差（如随机森林）")
print("   ⚡ Boosting: 适合提升弱模型性能（如 XGBoost）")
print("   🎯 Blending: 适合组合不同类型的模型")

### 4.3 模型持久化与部署

In [ ]:
# ============================================================
# 模型保存与加载
# PyCaret 提供了便捷的模型保存功能
# ============================================================

# 1. 保存模型（PyCaret 方式）
print("💾 保存模型...")
save_model(tuned_model, 'pycaret_best_model')
print("✅ 模型已保存为: pycaret_best_model.pkl\n")

# 2. 加载模型
print("📂 加载模型...")
loaded_model = load_model('pycaret_best_model')
print("✅ 模型加载成功\n")

# 3. 使用加载的模型预测
print("🔮 使用加载的模型进行预测...")
new_predictions = predict_model(loaded_model, data=test_data)
loaded_accuracy = accuracy_score(y_test, new_predictions['prediction_label'])
print(f"📊 加载模型的准确率: {loaded_accuracy:.4f}")
print(f"✅ 与原模型一致: {loaded_accuracy == accuracy}\n")

# 4. 导出为 Pickle（标准 Python 方式）
import pickle

with open('standard_model.pkl', 'wb') as f:
    pickle.dump(tuned_model, f)
print("✅ 也可使用标准 pickle 保存")

print("\n💡 模型部署建议:")
print("   1️⃣  本地部署：直接使用 .pkl 文件")
print("   2️⃣  Web服务：使用 Flask/FastAPI 封装 API")
print("   3️⃣  云部署：使用 AWS SageMaker、Azure ML等")
print("   4️⃣  边缘部署：转换为 ONNX、TensorFlow Lite等格式")

---

## 5. 其他 AutoML 工具介绍

### 5.1 TPOT（基于遗传算法）

In [ ]:
# ============================================================
# TPOT (Tree-based Pipeline Optimization Tool)
# 
# 特点：
# - 使用遗传算法（Genetic Programming）优化整个ML流程
# - 不仅优化超参数，还优化特征工程和模型选择
# - 轻量级，易于安装
# - 可导出为 sklearn Pipeline 代码
# ============================================================

# 安装: pip install tpot

# 示例代码（如果已安装TPOT，可取消注释运行）
"""
from tpot import TPOTClassifier

# 创建 TPOT 分类器
# generations: 遗传算法的代数
# population_size: 每代的种群大小
# cv: 交叉验证折数
tpot = TPOTClassifier(
    generations=5,       # 进化5代
    population_size=20,  # 每代20个个体
    cv=5,
    random_state=42,
    verbosity=2
)

# 训练（会自动搜索最佳pipeline）
tpot.fit(X_train, y_train)

# 评估
print(f'TPOT 准确率: {tpot.score(X_test, y_test):.4f}')

# 导出最佳 pipeline 为 Python 代码
tpot.export('tpot_best_pipeline.py')
"""

print("📚 TPOT 核心概念:")
print("\n1️⃣  遗传算法（Genetic Algorithm）:")
print("   - 初始化：随机生成一批 ML pipelines")
print("   - 选择：评估每个 pipeline 的性能")
print("   - 交叉：组合两个优秀 pipeline 的特征")
print("   - 变异：随机修改 pipeline 的某些部分")
print("   - 迭代：重复上述过程，逐步进化")
print("\n2️⃣  优势:")
print("   ✅ 自动化特征工程")
print("   ✅ 搜索空间广（包含整个 sklearn 生态）")
print("   ✅ 可导出代码，便于理解和定制")
print("\n3️⃣  适用场景:")
print("   - 数据集较小（< 100k 样本）")
print("   - 需要探索多种特征工程方法")
print("   - 想要理解 AutoML 的内部逻辑")

### 5.2 AutoML 工具对比总结

In [ ]:
# ============================================================
# 主流 AutoML 工具对比
# ============================================================

comparison_data = {
    '工具': ['Auto-sklearn', 'PyCaret', 'TPOT', 'H2O AutoML', 'FLAML'],
    '优化算法': ['贝叶斯优化', '网格/贝叶斯', '遗传算法', '随机搜索', 'CFO算法'],
    'macOS支持': ['❌', '✅', '✅', '✅', '✅'],
    '易用性': ['⭐⭐⭐', '⭐⭐⭐⭐⭐', '⭐⭐⭐⭐', '⭐⭐⭐⭐', '⭐⭐⭐'],
    '功能丰富度': ['⭐⭐⭐⭐', '⭐⭐⭐⭐⭐', '⭐⭐⭐', '⭐⭐⭐⭐⭐', '⭐⭐⭐⭐'],
    '性能': ['⭐⭐⭐⭐⭐', '⭐⭐⭐⭐', '⭐⭐⭐', '⭐⭐⭐⭐', '⭐⭐⭐⭐⭐'],
    '推荐场景': [
        'Linux研究/竞赛',
        '快速原型开发',
        '学习AutoML原理',
        '大规模数据',
        '高效优化'
    ]
}

df_comparison = pd.DataFrame(comparison_data)

print("="*80)
print("AutoML 工具对比表")
print("="*80)
print(df_comparison.to_string(index=False))
print("="*80)

print("\n💡 选择建议:")
print("\n🎯 初学者/快速开发 → PyCaret")
print("   - 最易上手，文档丰富")
print("   - 可视化功能强大")
print("   - 支持所有平台")

print("\n🧪 学习AutoML原理 → TPOT")
print("   - 可导出代码，便于理解")
print("   - 遗传算法直观")
print("   - 轻量级，易调试")

print("\n🏆 追求极致性能 → Auto-sklearn (Linux) / FLAML (跨平台)")
print("   - Auto-sklearn: 贝叶斯优化 + 元学习")
print("   - FLAML: 微软开发，高效搜索")

print("\n🏢 生产环境/大数据 → H2O AutoML")
print("   - 支持分布式训练")
print("   - 企业级功能")
print("   - Java/Python 双接口")

---

## 6. 总结与最佳实践

### 📚 本节核心知识点

#### 1. AutoML 的三大核心技术

**贝叶斯优化（Bayesian Optimization）**
```
工作原理：
1. 建立代理模型（Surrogate Model）预测参数性能
2. 使用采集函数（Acquisition Function）平衡探索与利用
3. 选择下一组最有希望的参数进行实验
4. 更新代理模型，重复迭代

优势：比随机搜索和网格搜索更高效
适用：连续参数空间，昂贵的评估函数
```

**元学习（Meta-learning）**
```
核心思想：从历史数据集中学习经验

工作流程：
1. 分析新数据集的特征（样本数、特征数、分布等）
2. 在元知识库中查找相似数据集
3. 推荐在相似数据集上表现好的模型
4. 从推荐的起点开始优化（而非随机初始化）

优势：快速找到好的起点，加速收敛
```

**自动集成（Ensemble）**
```
集成策略：
- Bagging: 降低方差（如随机森林）
- Boosting: 降低偏差（如XGBoost、AdaBoost）
- Stacking/Blending: 组合不同算法

优势：比单一模型更稳定、更准确
```

---

### 🎯 AutoML 使用最佳实践

#### 时间预算设置

```python
# 快速验证（1-3分钟）- 适合探索阶段
setup(..., session_id=42)
best = compare_models(fold=3)  # 减少交叉验证折数

# 标准训练（10-30分钟）- 适合一般项目
best = compare_models(fold=5)
tuned = tune_model(best, n_iter=10)

# 深度搜索（1-2小时）- 适合竞赛/重要项目
best = compare_models(fold=10)
tuned = tune_model(best, n_iter=50)
ensemble = blend_models([...])
```

#### 数据预处理建议

```python
# PyCaret 会自动处理很多预处理步骤，但仍需注意：

# 1. 检查数据质量
# - 缺失值比例
# - 类别不平衡
# - 异常值

# 2. 特征工程（可选）
# PyCaret 可以自动完成，但手动特征工程往往更好
setup(
    data=data,
    target='target',
    normalize=True,              # 标准化数值特征
    normalize_method='zscore',   # 使用 Z-score
    feature_selection=True,      # 特征选择
    remove_multicollinearity=True,  # 去除多重共线性
    multicollinearity_threshold=0.9
)
```

#### 模型评估指标选择

```python
# 分类任务
- 平衡数据集 → Accuracy
- 不平衡数据集 → F1-score、AUC
- 关注假阳性 → Precision
- 关注假阴性 → Recall

# 回归任务
- 一般情况 → R²、RMSE
- 对异常值敏感 → MAE
- 百分比误差 → MAPE
```

---

### ⚠️ 常见陷阱与注意事项

#### 1️⃣ 数据泄露（Data Leakage）

```python
# ❌ 错误做法：在划分训练/测试集之前进行特征工程
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # 在全部数据上fit
X_train, X_test = train_test_split(X_scaled, ...)

# ✅ 正确做法：让 AutoML 工具自动处理
# PyCaret 会在 setup() 时正确处理预处理流程
```

#### 2️⃣ 过拟合风险

```python
# AutoML 可能在小数据集上过拟合

# 防止过拟合的方法：
1. 使用更多的交叉验证折数（fold=10）
2. 在独立测试集上验证
3. 使用正则化模型
4. 限制模型复杂度
```

#### 3️⃣ 时间预算陷阱

```python
# ❌ 时间设置过短 → 搜索不充分
compare_models(fold=2)  # 太少，结果不稳定

# ❌ 时间设置过长 → 回报递减
tune_model(best, n_iter=1000)  # 后期提升微小

# ✅ 平衡效率与性能
compare_models(fold=5)
tune_model(best, n_iter=10-20)
```

#### 4️⃣ 盲目信任 AutoML

```python
# AutoML 不是万能的！

仍需要：
✅ 理解数据的业务含义
✅ 手动进行领域相关的特征工程
✅ 检查模型的可解释性
✅ 在真实场景中验证
✅ 监控模型在生产环境的表现
```

---

### 📊 AutoML 工作流程总结

```
1️⃣ 数据准备
   ├─ 探索性数据分析（EDA）
   ├─ 数据清洗
   └─ 划分训练/测试集

2️⃣ AutoML 初始化
   ├─ setup() 配置环境
   └─ 设置预处理选项

3️⃣ 模型搜索
   ├─ compare_models() 对比所有算法
   └─ 选择最佳模型

4️⃣ 超参数调优
   └─ tune_model() 优化性能

5️⃣ 模型集成（可选）
   ├─ ensemble_model() 单模型集成
   └─ blend_models() 多模型融合

6️⃣ 评估与部署
   ├─ predict_model() 测试集评估
   ├─ plot_model() 可视化分析
   └─ save_model() 保存模型
```

---

### 🎯 学习建议

1. **理论先行**：理解贝叶斯优化、元学习、集成学习的原理
2. **工具对比**：尝试不同 AutoML 工具，了解各自优势
3. **手动对照**：将 AutoML 结果与手动调参对比
4. **源码阅读**：查看 AutoML 导出的代码，学习最佳实践
5. **实战应用**：在真实项目中应用，积累经验

---

## 🎯 练习任务

### 任务 1：基础应用
使用 PyCaret 在 sklearn 的 Iris 数据集上完成完整的 AutoML 流程。

### 任务 2：对比实验
在同一数据集上对比以下三种方法的效果：
- 手动调参的单一模型
- PyCaret 的自动调优
- PyCaret 的模型集成

### 任务 3：理解原理
阅读 PyCaret 或 TPOT 导出的代码，分析：
- 选择了哪些特征工程方法？
- 最佳模型的超参数是什么？
- 为什么这些参数是最优的？

### 任务 4：自定义数据集
在你自己的数据集上使用 AutoML：
- 记录不同时间预算下的性能
- 尝试手动特征工程能否超越 AutoML
- 分析 AutoML 的优势和局限

---

## 📚 参考资源

### 官方文档
- [PyCaret 文档](https://pycaret.org/)
- [Auto-sklearn 文档](https://automl.github.io/auto-sklearn/)
- [TPOT 文档](http://epistasislab.github.io/tpot/)

### 学术论文
- [Auto-sklearn 论文](https://papers.nips.cc/paper/2015/hash/11d0e6287202fced83f79975ec59a3a6-Abstract.html)
- [贝叶斯优化综述](https://arxiv.org/abs/1807.02811)
- [元学习综述](https://arxiv.org/abs/1810.03548)

### 视频教程
- PyCaret YouTube 频道
- Kaggle AutoML 教程

---

**下一节预告**: `03_advanced_automl.ipynb` - 高级 AutoML 技巧与优化策略

**建议学习时间**: 2-3小时（理论 + 实践）